<a href="https://colab.research.google.com/github/AnonymousBazinga/IPL_ScorePredictor/blob/main/FinalIPLScore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
!pip install --quiet streamlit
!pip install --upgrade --quiet gradio

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from math import floor
from bs4 import BeautifulSoup
import requests
import re
import streamlit as st
import gradio as gr

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
path = '/content/drive/MyDrive/EXTRAS/triale.csv'
dataset = pd.read_csv(path)
x = dataset.drop(columns = ["Output","MatchID","StadiumID"])
y = dataset["Output"]
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2)

In [8]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(256, input_shape = x_train.shape[1:], activation = 'sigmoid'))
model.add(tf.keras.layers.Dense(64, activation = 'relu'))
model.add(tf.keras.layers.Dense(256, activation = 'softmax'))


In [9]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy', metrics = ['accuracy'])
model.fit(x_train, y_train, epochs=50)

Epoch 1/50
445/445 [==============================] - 2s 3ms/step - loss: 3.8823 - accuracy: 0.0439
Epoch 2/50
445/445 [==============================] - 1s 3ms/step - loss: 3.5949 - accuracy: 0.0682
Epoch 3/50
445/445 [==============================] - 2s 3ms/step - loss: 3.4614 - accuracy: 0.0789
Epoch 4/50
445/445 [==============================] - 1s 3ms/step - loss: 3.4095 - accuracy: 0.0868
Epoch 5/50
445/445 [==============================] - 1s 3ms/step - loss: 3.3461 - accuracy: 0.0930
Epoch 6/50
445/445 [==============================] - 1s 3ms/step - loss: 3.2557 - accuracy: 0.1014
Epoch 7/50
445/445 [==============================] - 1s 3ms/step - loss: 3.1580 - accuracy: 0.1082
Epoch 8/50
445/445 [==============================] - 1s 2ms/step - loss: 3.0843 - accuracy: 0.1146
Epoch 9/50
445/445 [==============================] - 1s 3ms/step - loss: 3.0185 - accuracy: 0.1213
Epoch 10/50
445/445 [==============================] - 2s 4ms/step - loss: 2.9785 - accuracy: 0.1291

In [10]:
model.evaluate(x_test,y_test)

112/112 [==============================] - 0s 2ms/step - loss: 2.4450 - accuracy: 0.2245


[2.445012092590332, 0.22453573346138]

In [11]:
def stadium_avg(stadium_name):
  url = "https://t20-head-to-head.com/statistics-by-ipl-venue/#"
  page = requests.get(url).text
  doc = BeautifulSoup(page,"html.parser")

  temp = 0
  page_text2 = doc.find_all("figure", attrs={'class':'wp-block-table'})
  page_text = page_text2[::2]
  for i in page_text:
    if stadium_name in str(i):
      temp = str(i.table.tbody).split('td')[9]

  stadium_avg = int(temp[1:4])
  return stadium_avg

stadium_avg("Narendra Modi")

169

In [18]:
def fn1(stadium_name, hometeam, runs, overs, wickets):
  avg_score = stadium_avg(stadium_name)
  arr = [[avg_score,hometeam, runs,overs,wickets]]
  t1 = model.predict(arr)
  a = list(t1[0])
  #print(a)
  runrate = 0
  for i in a:
    runrate = runrate+(a.index(i)*i)
  out2 = runrate*20
  out = floor(out2)
  return out//10

In [19]:
runs = int(st.number_input("How many runs scored so far?", min_value=0, max_value=256))
overs = float(st.select_slider("How many overs done so far?", options=[str(i)+"."+str(j) for i in range(0,20,1) for j in range(0,6,1)]))
wickets = int(st.select_slider("How many wickets lost far?", options=[i for i in range(0,11,1)]))
hometeam = st.radio("Does the batting team have home court advantage?", ("Yes", "No"))
stadium_name = str(st.text_input("Which stadium is the match being played in?"))
stadium_average = str(stadium_avg(stadium_name))
if hometeam == "Yes":
  hometeam = 1
else:
  hometeam = 0
text = fn1(stadium_name, hometeam, runs, overs, wickets)
st.write("The AI predicted score for this match is", text)

1/1 [==============================] - 0s 159ms/step


In [16]:
hometeam = gr.Radio([0,1], label="Does the batting team have home court advantage? (0=no,1=yes)")
runs = gr.Number(label="How many runs scored? (e.g. 60, 204)")
overs = gr.Number(label="How many overs finished? (e.g. 10.2, 3, 12.4)")
wickets = gr.Slider(minimum=0,maximum=10,step=1,label="How many wickets are lost?")
out = gr.Number(label="The AI predicted score for this innings is: ")
demo = gr.Interface(fn1,inputs = ['text',hometeam,runs,overs,wickets],outputs=out,theme='glass')
demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://7c8cd01ed289f7c200.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
